In [ ]:
!git clone https://token@github.com/TableASCII/RAG_evaluation

Cloning into 'RAG_evaluation'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 35 (delta 4), reused 14 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (35/35), 415.37 KiB | 2.21 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [ ]:
%cd /content/RAG_evaluation/RAG_EVAL

/content/RAG_evaluation/RAG_EVAL


In [ ]:
%pwd

'/content/RAG_evaluation/RAG_EVAL'

In [ ]:
from datasets import load_dataset
#ru_data = load_dataset("ai-forever/hist-rag-bench-public-texts", split="train", streaming=True)
ru_data = load_dataset("sberquad", split="train", streaming=True)

In [ ]:
import pandas as pd

# Используем set для хранения только уникальных текстов
unique_texts = set()

for entry in ru_data:
    text = entry["context"].strip()
    unique_texts.add(text)


# Превращаем в список словарей для DataFrame
raw_texts = [{"text": t} for t in unique_texts]

# Создаем DataFrame и перемешиваем
df = pd.DataFrame(raw_texts).sample(frac=1, random_state=42)
df.to_json("raw_texts.jsonl", orient="records", lines=True, force_ascii=False)

print(f"Готово! Файл создан. Всего уникальных записей: {len(df)}")

Готово! Файл создан. Всего уникальных записей: 9078


In [ ]:
from google.colab import userdata
API_KEY= userdata.get('API_KEY7')

In [ ]:
import json
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=API_KEY,
)

In [ ]:
import sqlite3
import json
from openai import OpenAI

# Настройка БД
conn = sqlite3.connect('rag_eval.db')
cursor = conn.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS statements_data
                  (id INTEGER PRIMARY KEY, source_text TEXT UNIQUE, statements_json TEXT)''')
conn.commit()

def get_statements(source_text):
    system_instruction = (
        "Ты — эксперт по извлечению фактов. Твоя цель — разбить входной текст на атомарные, "
        "независимые утверждения. Важно: заменяй все местоимения (он, она, оно, они и т. д.) "
        "на конкретные субъекты, к которым они относятся. Каждое утверждение должно быть самодостаточным. "
        "ГЛАВНЫЕ ПРАВИЛА (Разрешение анафор и указателей):"
        "Заменяй все личные местоимения (он, она, они, их)."
        "Критически важно: Заменяй все указательные местоимения и отсылки (это, то, об этом, из-за этого, таким образом)."
        "Вместо слов «это» или «об этом» вставляй полное описание события или объекта, о котором идет речь."
        "Каждое утверждение должно быть понятно человеку, который не видел исходный текст. Никакой двусмысленности."
        "СИНТАКСИЧЕСКАЯ ПРОСТОТА: Используй только прямой порядок слов: [Субъект] -> [Действие] -> [Объект]."
        "Избегай сложных придаточных предложений (со слов \"который\", \"о том что\")."
        "УДАЛЕНИЕ ШУМА: Выбрасывай вводные слова (\"обычно\", \"вероятно\", \"по мнению\", \"интегрально\"), если они не меняют суть факта."
        "Факт должен быть \"сухим\" утверждением."
        "Пример: Вход: «Илон Маск купил Twitter. Он долго думал об этом, но все же сделал это.» "
        "Плохо: «Илон Маск долго думал об этом», «Илон Маск сделал это». "
        "Хорошо: «Илон Маск долго думал о покупке Twitter», «Илон Маск совершил сделку по покупке Twitter»."
        "Возвращай результат строго в виде JSON-объекта с ключом 'facts', содержащим список строк. "
        "Язык ответа должен соответствовать языку входного текста."
    )

    completion = client.chat.completions.create(
        model="z-ai/glm-4.5-air:free",
        messages=[
            {"role": "system", "content": system_instruction},
            {"role": "user", "content": source_text}
        ],
        response_format={"type": "json_object"}
    )

    # Возвращаем только содержимое сообщения (JSON-строку)
    return completion.choices[0].message.content
i = 0
# Цикл обработки
for item in raw_texts[:50]:
    text = item['text']

    i+=1

    # ШАГ 1: Проверяем наличие текста в БД перед вызовом функции
    cursor.execute("SELECT 1 FROM statements_data WHERE source_text = ?", (text,))
    exists = cursor.fetchone()

    if exists:
        print(f"Пропуск: текст уже обработан и есть в базе.")
        continue # Переходим к следующему итератору, get_statements() не вызывается

    # ШАГ 2: Если текста нет, вызываем LLM и сохраняем
    try:
        print(f"Обработка нового текста...")
        result_json = get_statements(text)

        # Валидация JSON
        json.loads(result_json)

        cursor.execute("INSERT INTO statements_data (source_text, statements_json) VALUES (?, ?)",
                       (text, result_json))
        conn.commit()
        print(f"{i}: Данные успешно сохранены.")
    except Exception as e:
        print(f"Ошибка при обработке: {e}")

# Закрываем соединение в самом конце
conn.close()
print("Сбор данных завершен.")

Пропуск: текст уже обработан и есть в базе.
Пропуск: текст уже обработан и есть в базе.
Пропуск: текст уже обработан и есть в базе.
Пропуск: текст уже обработан и есть в базе.
Пропуск: текст уже обработан и есть в базе.
Пропуск: текст уже обработан и есть в базе.
Пропуск: текст уже обработан и есть в базе.
Пропуск: текст уже обработан и есть в базе.
Обработка нового текста...
9: Данные успешно сохранены.
Пропуск: текст уже обработан и есть в базе.
Пропуск: текст уже обработан и есть в базе.
Пропуск: текст уже обработан и есть в базе.
Пропуск: текст уже обработан и есть в базе.
Пропуск: текст уже обработан и есть в базе.
Пропуск: текст уже обработан и есть в базе.
Обработка нового текста...
16: Данные успешно сохранены.
Пропуск: текст уже обработан и есть в базе.
Пропуск: текст уже обработан и есть в базе.
Пропуск: текст уже обработан и есть в базе.
Пропуск: текст уже обработан и есть в базе.
Пропуск: текст уже обработан и есть в базе.
Пропуск: текст уже обработан и есть в базе.
Обработ

Скачивание БД

In [ ]:
from google.colab import files

# Укажите имя вашего файла, который лежит в корневой папке Колаба
filename = '/content/rag_eval.db'

files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Удаление записи из БД

In [ ]:
import sqlite3

conn = sqlite3.connect('rag_eval.db')
cursor = conn.cursor()

# Удаляем строку, где id равен 5
target_id = 85
cursor.execute("DELETE FROM statements_data WHERE id = ?", (target_id,))

conn.commit() # Не забывайте сохранять изменения!
print(f"Запись с id {target_id} удалена.")

Запись с id 85 удалена.


Получение записи из БД

In [ ]:
import sqlite3
import json

# Подключаемся
conn = sqlite3.connect('rag_eval.db')
cursor = conn.cursor()

# ID записи, которую хотим получить
target_id = 87

# Выполняем запрос
cursor.execute("SELECT id, source_text, statements_json FROM statements_data WHERE id = ?", (target_id,))

# Извлекаем результат
row = cursor.fetchone()

if row:
    print(f"ID: {row[0]}")
    print(f"Текст: {row[1]}")
    # Если в statements_json хранится строка-JSON, ее можно распарсить обратно в словарь/список
    data = json.loads(row[2])
    print(f"JSON данные: {data}")
else:
    print(f"Запись с id {target_id} не найдена.")

conn.close()

ID: 87
Текст: В профессиональных оценках риска, риск обычно комбинирует вероятность наступающего события с воздействием, которое оно могло бы произвести, а также с обстоятельствами, сопровождающими наступление этого события. Однако там, где активы оцениваются рынком, вероятности и воздействия всех событий интегрально отражаются в рыночной цене, и риск поэтому наступает только от изменения этой цены; это — одно из следствий теории оценивания Блэка-Шоулса. С точки зрения RUP (Rational Unified Process) риск — действующий/развивающийся фактор процесса, обладающий потенциалом негативного влияния на ход процесса.
JSON данные: {'facts': ['В профессиональных оценках риска риск обычно комбинирует вероятность наступающего события с воздействием, которое событие могло бы произproduce.', 'В профессиональных оценках риск комбинирует вероятность наступающего события с обстоятельствами, сопровождающими наступление события.', 'В оценках активов, которые осуществляются рынком, вероятности и воздействия

Было / стало

ID: 85
Текст: В профессиональных оценках риска, риск обычно комбинирует вероятность наступающего события с воздействием, которое оно могло бы произвести, а также с обстоятельствами, сопровождающими наступление этого события. Однако там, где активы оцениваются рынком, вероятности и воздействия всех событий интегрально отражаются в рыночной цене, и риск поэтому наступает только от изменения этой цены; это — одно из следствий теории оценивания Блэка-Шоулса. С точки зрения RUP (Rational Unified Process) риск — действующий/развивающийся фактор процесса, обладающий потенциалом негативного влияния на ход процесса.
JSON данные:

{'facts': ['В профессиональных оценках риска риск обычно комбинирует вероятность наступающего события с воздействием, которое это событие могло бы произвести, а также с обстоятельствами, сопровождающими наступление этого события.', 'Однако там, где активы оцениваются рынком, вероятности и воздействия всех событий интегрально отражаются в рыночной цене, и риск поэтому наступает только от изменения рыночной цены.', 'Утверждение о том, что риск наступает только от изменения рыночной цены является одним из следствий теории оценивания Блэка-Шоулса.', 'С точки зрения RUP (Rational Unified Process) риск является действующим/развивающимся фактором процесса, обладающим потенциалом негативного влияния на ход процесса.']}

JSON данные: {'facts': ['В профессиональных оценках риска риск обычно комбинирует вероятность наступающего события с воздействием, которое событие могло бы произproduce.', 'В профессиональных оценках риск комбинирует вероятность наступающего события с обстоятельствами, сопровождающими наступление события.', 'В оценках активов, которые осуществляются рынком, вероятности и воздействия всех событий отражаются в рыночной цене.', 'Риск в случаях, когда активы оцениваются рынком, возникает только от изменения рыночной цены.', 'Утверждение о том, что риск в случаях, когда активы оцениваются рынком, возникает только от изменения рыночной цены, является одним из следствий теории оценивания Блэка-Шоулса.', 'С точки зрения RUP (Rational Unified Process) риск является действующим/развивающимся фактором процесса.', 'Риск обладает потенциалом негативного влияния на ход процесса с точки зрения RUP (Rational Unified Process).']}


Получение jsonl из БД

In [ ]:
import sqlite3
import json

def export_to_jsonl(db_path, output_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Выбираем все данные
    cursor.execute("SELECT source_text, statements_json FROM statements_data")
    rows = cursor.fetchall()

    with open(output_path, 'w', encoding='utf-8') as f:
        for source_text, statements_json in rows:
            try:
                # 1. Превращаем строку из БД в Python-словарь
                extracted_data = json.loads(statements_json)

                # 2. Формируем финальную структуру строки датасета
                # Мы берем список фактов из ключа 'facts'
                line_to_write = {
                    "text": source_text,
                    "facts": extracted_data.get("facts", [])
                }

                # 3. Сериализуем обратно в строку и записываем
                # ensure_ascii=False сохраняет кириллицу
                f.write(json.dumps(line_to_write, ensure_ascii=False) + '\n')

            except Exception as e:
                print(f"Ошибка парсинга для текста: {source_text[:30]}... Ошибка: {e}")

    conn.close()
    print(f"Готово! Файл сохранен: {output_path}")

# Запуск
export_to_jsonl('rag_eval.db', 'dataset_final.jsonl')
files.download('dataset_final.jsonl')

Готово! Файл сохранен: dataset_final.jsonl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>